In [37]:
from __future__ import print_function
import sys, os
import h2o
from h2o.estimators.deepwater import H2ODeepWaterEstimator

import importlib
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,01 secs
H2O cluster version:,3.11.0.99999
H2O cluster version age:,1 hour and 10 minutes
H2O cluster name:,arno
H2O cluster total nodes:,1
H2O cluster free memory:,13.96 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [38]:
if not H2ODeepWaterEstimator.available(): quit()

# LeNET

Here we define the famous LENET neural network, but you can define any deep neural network of your choice.

In [39]:
def lenet(num_classes):
    import mxnet as mx
    data = mx.symbol.Variable('data')
    # first conv
    conv1 = mx.symbol.Convolution(data=data, kernel=(5,5), num_filter=20)
    tanh1 = mx.symbol.Activation(data=conv1, act_type="tanh")
    pool1 = mx.symbol.Pooling(data=tanh1, pool_type="max", kernel=(2,2), stride=(2,2))
    # second conv
    conv2 = mx.symbol.Convolution(data=pool1, kernel=(5,5), num_filter=50)
    tanh2 = mx.symbol.Activation(data=conv2, act_type="tanh")
    pool2 = mx.symbol.Pooling(data=tanh2, pool_type="max", kernel=(2,2), stride=(2,2))
    # first fullc
    flatten = mx.symbol.Flatten(data=pool2)
    fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=500)
    tanh3 = mx.symbol.Activation(data=fc1, act_type="tanh")
    # second fullc
    fc2 = mx.symbol.FullyConnected(data=tanh3, num_hidden=num_classes)
    # loss
    lenet = mx.symbol.SoftmaxOutput(data=fc2, name='softmax')
    return lenet

## DeepWater for MXNET

We can use the import functions provided by h2o to import the list of 

In [40]:
train = h2o.import_file("../../bigdata/laptop/mnist/train.csv.gz")
test = h2o.import_file("../../bigdata/laptop/mnist/test.csv.gz")

predictors = list(range(0,784))
resp = 784
train[resp] = train[resp].asfactor()
test[resp] = test[resp].asfactor()
nclasses = train[resp].nlevels()[0]

Parse progress: |█████████████████████████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████████████████████████| 100%


Let's create the lenet model architecture from scratch using the MXNet Python API

In [41]:
model = lenet(nclasses)

To import the model inside the DeepWater training engine we need to save the model to a file:

In [42]:
model_path = "/tmp/symbol_lenet-py.json"
model.save(model_path)

The model is just the structure of the network expressed as a json dict

In [43]:
#!head "/tmp/symbol_lenet-py.json"

{
  "nodes": [
    {
      "op": "null", 
      "param": {}, 
      "name": "data", 
      "inputs": [], 
      "backward_source_id": -1
    }, 
    {


## Importing the LeNET model architecture for training in H2O

We have defined the model and saved the structure to a file. We are ready to start the training procedure.

In [44]:
model = H2ODeepWaterEstimator(epochs=100, learning_rate=1e-3, 
                              mini_batch_size=64,
                              network='user', 
                              network_definition_file=model_path,
                              image_shape=[28,28], channels=1)

In [45]:
model.train(x=predictors,y=resp, training_frame=train, validation_frame=test)

deepwater Model Build progress: |█████████████████████████████████████████████████████████████| 100%


In [46]:
model.show()

Model Details
H2ODeepWaterEstimator :  Deep Water
Model Key:  DeepWater_model_python_1476000466538_1
Status of Deep Learning Model: user: 1.6 MB, predicting C785, 10-class classification, 6,000,640 training samples, mini-batch size 64



,rate,momentum
,0.0001428,0.99




ModelMetricsMultinomial: deepwater
** Reported on train data. **

MSE: 0.00412073914553
RMSE: 0.064192983616
LogLoss: 0.0182603796823
Mean Per-Class Error: 0.00350493808739
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
1011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"0 / 1,011"
1.0,1166.0,3.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0068143,"8 / 1,174"
0.0,2.0,963.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0031056,3 / 966
0.0,0.0,3.0,1002.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0059524,"6 / 1,008"
1.0,1.0,0.0,0.0,1010.0,0.0,0.0,0.0,0.0,2.0,0.0039448,"4 / 1,014"
0.0,0.0,0.0,0.0,0.0,897.0,0.0,0.0,1.0,0.0,0.0011136,1 / 898
0.0,0.0,0.0,0.0,0.0,1.0,977.0,0.0,0.0,0.0,0.0010225,1 / 978
0.0,0.0,0.0,1.0,1.0,0.0,0.0,1014.0,0.0,0.0,0.0019685,"2 / 1,016"
1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,989.0,1.0,0.0050302,5 / 994
1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,978.0,0.0060976,6 / 984


Top-10 Hit Ratios: 


...

Top-10 Hit Ratios: 


k,hit_ratio
1,0.987
2,0.9976000
3,0.9995999
4,0.9998999
5,0.9998999
6,0.9999999
7,0.9999999
8,0.9999999
9,0.9999999
10,0.9999999


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2016-10-09 01:07:54,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan
,2016-10-09 01:07:55,3.328 sec,5432 obs/sec,0.0341333,1,2048.0,0.6475480,1.1441473,0.3204222,0.6408030,1.1212354,0.3057
,2016-10-09 01:08:12,19.241 sec,22944 obs/sec,5.9733333,175,358400.0,0.1552907,0.0907716,0.0250921,0.1548827,0.0874353,0.026
,2016-10-09 01:08:19,26.007 sec,25087 obs/sec,9.0794667,266,544768.0,0.1369462,0.0715630,0.0191178,0.1412460,0.0720911,0.0215
,2016-10-09 01:08:25,32.809 sec,26146 obs/sec,12.1173333,355,727040.0,0.1267589,0.0613674,0.0169272,0.1344119,0.0644651,0.0207
---,---,---,---,---,---,---,---,---,---,---,---,---
,2016-10-09 01:11:15,3 min 22.201 sec,29064 obs/sec,87.3813333,2560,5242880.0,0.0673856,0.0197682,0.0042816,0.1049248,0.0379410,0.0131
,2016-10-09 01:11:22,3 min 29.136 sec,29101 obs/sec,90.5216,2652,5431296.0,0.0668086,0.0194800,0.0040824,0.1052558,0.0381017,0.0132
,2016-10-09 01:11:29,3 min 36.111 sec,29162 obs/sec,93.7642667,2747,5625856.0,0.0659256,0.0190596,0.0038833,0.1048264,0.0379124,0.013
,2016-10-09 01:11:36,3 min 43.049 sec,29175 obs/sec,96.8362667,2837,5810176.0,0.0649722,0.0186320,0.0037837,0.1046608,0.0376912,0.0128



See the whole table with table.as_data_frame()
